# **Feature Extraction Methods: Imbalanced Data Without Annotations**

- *Key Features*: [MFCCs, Mel-Spectrograms, Chroma Frequencies, RMS Power]
- *Key Manipulations*: [Varying Window Sizes, Normalization, Average Pooling (Compression), Filtering]
- *Process Assistence*: [Converting them to numpy arrays now, easy label access across features]
- *Conversion*: [To numpy arrays and pkl files]


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time

# Libraries for audio
from IPython.display import Audio
import librosa
import librosa.display

# Training and Testing Split
from sklearn.model_selection import train_test_split

# for normalization & avgpooling features
import tensorflow as tf
# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random
import IPython.display as ipd
from functools import partial

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Variabels to be reused
path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/audio_files' 
npy_path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train_audio_npy/' 
train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train-not-annotated.csv' 
annotated_train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train-annotated.csv'
not_annotated_splt = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/trainval-split/trainval.csv'
sr = 22050

In [3]:
trainval_data = pd.read_csv(not_annotated_splt)
train_data = trainval_data[trainval_data['set'] == 'tr']
val_data = trainval_data[trainval_data['set'] == 'val']

In [15]:
train_data['species'].value_counts()

species
Alcedo atthis                39
Himantopus himantopus        36
Porphyrio porphyrio          33
Acrocephalus melanopogon     32
Gallinula chloropus          32
Fulica atra                  28
Tachybaptus ruficollis       27
Botaurus stellaris           27
Charadrius alexandrinus      26
Ardea purpurea               26
Dendrocopos minor            25
Motacilla flava              24
Acrocephalus arundinaceus    23
Ciconia ciconia              22
Ixobrychus minutus           21
Acrocephalus scirpaceus      20
Circus aeruginosus           19
Coracias garrulus            11
Anas platyrhynchos            9
Anas strepera                 5
Name: count, dtype: int64

# **Creating a class to do the extraction**

In [6]:
class Extraction:
  def __init__(self, train_df, val_df, window_size, overlap=0.5, npy_path=npy_path, sr=sr, n_mels=60, n_mfcc=20, n_chroma=12, features=['mfcc'], normalize=True, avgpool=False):
    """
    Instantiate the Extraction class to extract features.

    Parameters:
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'rms', 'melspectrogram'.
      normalize (bool): Whether to normalize the features.
      maxpool (bool): Whether to maxpool the features.
    """
    
    self.train_df = train_df
    self.val_df = val_df
    self.npy_path = npy_path
    self.window_size = window_size
    self.overlap = overlap
    self.sr = sr
    self.n_mels = n_mels
    self.n_mfcc = n_mfcc
    self.n_chroma = n_chroma

     # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    self.accepted_feature = ['mfcc', 'chroma', 'rms', 'melspectrogram']
    for feature in self.features:
      assert feature in self.accepted_feature, f"{feature} is not an accepted feature, only 'mfcc', 'chroma', 'rms', 'melspectrogram' are accepted features."

    self.normalize = normalize
    self.avgpool = avgpool

    print(f"Train DataFrame shape: {train_df.shape}")
    print(f"Validation DataFrame shape: {val_df.shape}")

    # extract train and val labels and features
    self.train_y, self.train_features = self.feature_extraction(self.train_df, window_size=self.window_size)
    self.val_y, self.val_features = self.feature_extraction(self.val_df, window_size=self.window_size)

    # process the features by average pooling
    self.train_features, self.val_features = self.process_features(self.train_features, self.val_features)

    
  

  def normalize_audio(self, audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))
  
  def bandpass_filter(self, audio, lowcut=800, highcut=8000, order=4):
    nyquist = 0.5 * self.sr  # Nyquist frequency
    low = lowcut / nyquist
    high = highcut / nyquist

    b, a = butter(order, [low, high], btype='band')
    filtered_audio = filtfilt(b, a, audio)
    return filtered_audio
  
  def generate_pink_noise(self, num_samples):
    white_noise = np.random.randn(num_samples)
    
    # Apply a filter to convert white noise into pink noise (1/f noise)
    X = np.fft.rfft(white_noise)
    S = np.arange(1, len(X) + 1)  # Frequency scaling
    pink_noise = np.fft.irfft(X / S)

    if len(pink_noise) < num_samples:
        # Pad with zeros if the length is less than num_samples
        pink_noise = np.pad(pink_noise, (0, num_samples - len(pink_noise)), mode='constant')
    elif len(pink_noise) > num_samples:
        # Trim if necessary
        pink_noise = pink_noise[:num_samples]
    
    return self.normalize_audio(pink_noise)
  
  def pad_with_noise(self, audio_data, window_length, window_samples):
    current_length = librosa.get_duration(y=audio_data, sr=self.sr)

    if current_length > window_length:
        return audio_data
    
    target_length_samples = int(window_length * sr) 
    current_length_samples = window_samples
    padding_length_samples = target_length_samples - current_length_samples

    assert target_length_samples == (current_length_samples+padding_length_samples)
    
    # Generate pink noise to pad with
    pink_noise = self.generate_pink_noise(padding_length_samples)
    padded_audio = np.concatenate([audio_data, pink_noise])
    # if len(padded_audio) < target_length_samples:
    #     padded_audio = np.append(padded_audio, self.generate_pink_noise(1))

    assert target_length_samples == len(padded_audio)
    
    return padded_audio
  
  def avg_pooling_keras(self, feature):
    # Clear the previous Keras session
    tf.keras.backend.clear_session()

    # Define the input shape based on features
    input_shape = feature.shape[1:]  # (n_mels, time_steps)

    # Create the Keras model for average pooling
    inputs = tf.keras.layers.Input(shape=input_shape)
    pooled = tf.keras.layers.GlobalAveragePooling1D()(inputs)
    pooling_model = tf.keras.models.Model(inputs=inputs, outputs=pooled)

    # Perform pooling using the model
    pooled_features = pooling_model.predict(feature)

    return pooled_features

#-------------------------Feature Extraction---------------------------------------
  def extract_mfcc(self, window):
    mfcc = librosa.feature.mfcc(y=window, sr=self.sr, n_mfcc=self.n_mfcc)
    if self.normalize:
      return librosa.util.normalize(mfcc)
    else:
      return mfcc


  def extract_chroma(self, window):
    chroma = librosa.feature.chroma_stft(y=window, sr=self.sr, n_chroma=self.n_chroma)
    if self.normalize:
      return librosa.util.normalize(chroma)
    else:
      return chroma
   

  def extract_rms(self, window):
    return librosa.feature.rms(y=window)

  def extract_melspectrogram(self, window):
    mel = librosa.feature.melspectrogram(y=window, sr=self.sr, n_mels=self.n_mels)
    if self.normalize:
      return librosa.util.normalize(mel)
    else:
      return mel
    
  def avgpooling(self, train_X, val_X, n_time, n_features):
    """
    Average pooling the train and val features.

    Parameters:
      train_X (npy): Training feature array of shape (batch_size, n_features, n_time)
      val_X (npy): Validation feature array of shape (batch_size, n_features, n_time)
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature array of shape (batch_size, n_features)
      val_X (npy): Avgpooled validation feature array of shape (batch_size, n_features)
    """
    # Clear the Keras session
    tf.keras.backend.clear_session()
    
    # Create the Keras input layer with shape (n_features, n_time)
    input_layer = tf.keras.layers.Input(shape=(n_features, n_time))
    
    # Apply average pooling over the time axis (axis=-1) to reduce n_time
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1))(input_layer)
    
    # Build the model
    pooling_model = tf.keras.models.Model(inputs=input_layer, outputs=avg_pool)

    # Use the model to apply average pooling on the training and validation features
    train_X = pooling_model.predict(train_X)
    val_X = pooling_model.predict(val_X)

    return train_X, val_X

    
  def process_features(self, train_features_dict, val_features_dict):
    for each in train_features_dict.keys():
      
      if each == 'mfcc':
        n_features=self.n_mfcc
      elif each == 'chroma':
        n_features=self.n_chroma
      elif each == 'rms':
        n_features=1
      elif each == 'melspectrogram':
        n_features=self.n_mels
      
      train_feature = train_features_dict[each]
      val_feature = val_features_dict[each]

      if self.avgpool:
        train_features_dict[each], val_features_dict[each] = self.avgpooling(train_feature, val_feature, n_time=train_feature.shape[2], n_features=n_features)
      else:
        train_features_dict[each], val_features_dict[each] = train_features_dict[each], val_features_dict[each]
    
    return train_features_dict, val_features_dict
      

  def feature_extraction(self, dataframe, window_size, filter=True):
    y = [] # To hold the labels
    features_dict = {item: [] for item in self.features} # Create a key for each feature listed
    print(f"Number of rows in dataframe: {len(dataframe)}")
    for _, row in tqdm(dataframe.iterrows(), desc="Processing data", total=len(dataframe)):
          label = row['species']
          file_path = os.path.join(self.npy_path, row['filename_npy'])

          try:
            
              audio = np.load(file_path)
          except FileNotFoundError:
              print(f"File not found: {file_path}")
              continue


          audio = self.normalize_audio(audio)

          audio = self.pad_with_noise(audio, window_length=self.window_size, window_samples=len(audio))
          # print(len(sample))

          if filter:
                  audio = self.bandpass_filter(audio)

          window_samples = int(window_size * self.sr)
          hop_samples = int(window_samples * (1 - self.overlap))  # For overlapping

          # Break the audio into windows with the specified overlap
          audio_windows = librosa.util.frame(audio, frame_length=window_samples, hop_length=hop_samples).T
          
          
          # display(label)
          
          for _, window in enumerate(audio_windows):
              
              y.append(label)

              if len(window) < window_samples:
                  if len(window) < 512*2:
                     continue
                  else:
                      window = self.pad_with_noise(window, window_length=window_size)
              
              # Feature Extraction FR --------------------------------------------------------------------
              # dynatically call the extract_x function to extract the listed features
              for feature in self.features:
                extract = f"extract_{feature}"
                if hasattr(self, extract) and callable(func := getattr(self, extract)):
                  features_dict[feature].append(func(window))

          # cast lists to np arrays
    for each in features_dict.keys():
              features_dict[each] = np.array(features_dict[each])

    y = np.array(y)

          # If not using average pooling, return resized features
    return y, features_dict

# **Average Pooling**

## **Window Size = 6s**

### **['melspectrogram']**

In [127]:
features_list = ['melspectrogram']

In [128]:
features = Extraction(train_data,
                      val_data,
                      window_size=6,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [01:10<00:00,  6.90it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:17<00:00,  7.42it/s]



223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 907us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step


In [129]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(7105,)

'melspectrogram'

(7105, 60)

In [130]:
train_y

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', ..., 'Anas strepera', 'Anas strepera',
       'Anas strepera'], dtype='<U25')

In [131]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(1896,)

'melspectrogram'

(1896, 60)

### Encode Classes

In [132]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [133]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

7105

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

1896

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [134]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [135]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[1.1545551e-05, 1.2176995e-05, 1.3009772e-05, ..., 4.4375400e-07,
          2.2230227e-08, 1.8851988e-08],
         [8.5422990e-08, 9.0774186e-08, 1.0014884e-07, ..., 1.3420139e-06,
          4.4213351e-07, 4.1396237e-07],
         [6.1857563e-06, 6.2330437e-06, 6.3144412e-06, ..., 1.1637738e-06,
          1.5971284e-07, 1.3454519e-07],
         ...,
         [7.1480241e-07, 7.4554231e-07, 7.9745979e-07, ..., 1.4340408e-06,
          1.3201277e-06, 1.2859776e-06],
         [6.0994887e-07, 6.2010758e-07, 6.3716243e-07, ..., 4.0761867e-07,
          1.7535218e-07, 1.6836647e-07],
         [6.8865182e-07, 6.9677486e-07, 7.1000841e-07, ..., 3.5803578e-06,
          1.9428686e-08, 1.4416809e-08]], dtype=float32),
  'label': array([1, 1, 1, ..., 5, 5, 5])},
 'val': {'melspectrogram': array([[2.08654342e-06, 2.09023460e-06, 2.09655309e-06, ...,
          4.35780862e-07, 2.86455418e-07, 2.62781299e-07],
         [5.14644796e-07, 5.18451770e-07, 5.24857455e-0

### Save the merged dictionary to a pkl

In [136]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_6s_mel.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc']**

In [137]:
features_list = ['melspectrogram', 'mfcc']

In [138]:
features = Extraction(train_data,
                      val_data,
                      window_size=6,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [03:49<00:00,  2.12it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:34<00:00,  3.73it/s]


223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step


In [139]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(7105,)

'melspectrogram'

(7105, 60)

'mfcc'

(7105, 20)

In [140]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(1896,)

'melspectrogram'

(1896, 60)

'mfcc'

(1896, 20)

### Encode Classes

In [141]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [142]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

7105

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

1896

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [143]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [144]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[1.1545551e-05, 1.2176995e-05, 1.3009772e-05, ..., 4.4375400e-07,
          2.2230227e-08, 1.8851988e-08],
         [8.5422990e-08, 9.0774186e-08, 1.0014884e-07, ..., 1.3420139e-06,
          4.4213351e-07, 4.1396237e-07],
         [6.1857563e-06, 6.2330437e-06, 6.3144412e-06, ..., 1.1637738e-06,
          1.5971284e-07, 1.3454519e-07],
         ...,
         [7.1480241e-07, 7.4554231e-07, 7.9745979e-07, ..., 1.4340408e-06,
          1.3201277e-06, 1.2859776e-06],
         [6.0994887e-07, 6.2010758e-07, 6.3716243e-07, ..., 4.0761867e-07,
          1.7535218e-07, 1.6836647e-07],
         [6.8865182e-07, 6.9677486e-07, 7.1000841e-07, ..., 3.5803578e-06,
          1.9428686e-08, 1.4416809e-08]], dtype=float32),
  'mfcc': array([[-1.00000000e+00, -2.42030799e-01, -4.80754524e-01, ...,
           3.10237170e-03,  1.18157947e-02, -6.41129969e-04],
         [-1.00000000e+00, -2.10041642e-01, -3.97155881e-01, ...,
          -2.89330794e-03,  1.39303915e-02, 

### Save the merged dictionary to a pkl

In [145]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_6s_mel_mfcc.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma']**

In [146]:
features_list = ['melspectrogram', 'mfcc', 'chroma']

In [147]:
features = Extraction(train_data,
                      val_data,
                      window_size=6,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [05:10<00:00,  1.56it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:01<00:00,  2.10it/s]


223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 469us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step


In [148]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(7105,)

'melspectrogram'

(7105, 60)

'mfcc'

(7105, 20)

'chroma'

(7105, 12)

In [149]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(1896,)

'melspectrogram'

(1896, 60)

'mfcc'

(1896, 20)

'chroma'

(1896, 12)

### Encode Classes

In [150]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [151]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

7105

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

1896

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [152]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [153]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[1.1545551e-05, 1.2176995e-05, 1.3009772e-05, ..., 4.4375400e-07,
          2.2230227e-08, 1.8851988e-08],
         [8.5422990e-08, 9.0774186e-08, 1.0014884e-07, ..., 1.3420139e-06,
          4.4213351e-07, 4.1396237e-07],
         [6.1857563e-06, 6.2330437e-06, 6.3144412e-06, ..., 1.1637738e-06,
          1.5971284e-07, 1.3454519e-07],
         ...,
         [7.1480241e-07, 7.4554231e-07, 7.9745979e-07, ..., 1.4340408e-06,
          1.3201277e-06, 1.2859776e-06],
         [6.0994887e-07, 6.2010758e-07, 6.3716243e-07, ..., 4.0761867e-07,
          1.7535218e-07, 1.6836647e-07],
         [6.8865182e-07, 6.9677486e-07, 7.1000841e-07, ..., 3.5803578e-06,
          1.9428686e-08, 1.4416809e-08]], dtype=float32),
  'mfcc': array([[-1.00000000e+00, -2.42030799e-01, -4.80754524e-01, ...,
           3.10237170e-03,  1.18157947e-02, -6.41129969e-04],
         [-1.00000000e+00, -2.10041642e-01, -3.97155881e-01, ...,
          -2.89330794e-03,  1.39303915e-02, 

### Save the merged dictionary to a pkl

In [154]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_6s_mel_mfcc_chroma.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma', 'rms']**

In [155]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'rms']

In [156]:
features = Extraction(train_data,
                      val_data,
                      window_size=6,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [05:59<00:00,  1.35it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:15<00:00,  1.73it/s]


223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 435us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 426us/step
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step


In [157]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(7105,)

'melspectrogram'

(7105, 60)

'mfcc'

(7105, 20)

'chroma'

(7105, 12)

'rms'

(7105, 1)

In [158]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(1896,)

'melspectrogram'

(1896, 60)

'mfcc'

(1896, 20)

'chroma'

(1896, 12)

'rms'

(1896, 1)

### Encode Classes

In [159]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [160]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

7105

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

1896

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [161]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [162]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[1.1545551e-05, 1.2176995e-05, 1.3009772e-05, ..., 4.4375400e-07,
          2.2230227e-08, 1.8851988e-08],
         [8.5422990e-08, 9.0774186e-08, 1.0014884e-07, ..., 1.3420139e-06,
          4.4213351e-07, 4.1396237e-07],
         [6.1857563e-06, 6.2330437e-06, 6.3144412e-06, ..., 1.1637738e-06,
          1.5971284e-07, 1.3454519e-07],
         ...,
         [7.1480241e-07, 7.4554231e-07, 7.9745979e-07, ..., 1.4340408e-06,
          1.3201277e-06, 1.2859776e-06],
         [6.0994887e-07, 6.2010758e-07, 6.3716243e-07, ..., 4.0761867e-07,
          1.7535218e-07, 1.6836647e-07],
         [6.8865182e-07, 6.9677486e-07, 7.1000841e-07, ..., 3.5803578e-06,
          1.9428686e-08, 1.4416809e-08]], dtype=float32),
  'mfcc': array([[-1.00000000e+00, -2.42030799e-01, -4.80754524e-01, ...,
           3.10237170e-03,  1.18157947e-02, -6.41129969e-04],
         [-1.00000000e+00, -2.10041642e-01, -3.97155881e-01, ...,
          -2.89330794e-03,  1.39303915e-02, 

### Save the merged dictionary to a pkl

In [163]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_6s_all.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

## **Window Size = 4s**

### **['melspectrogram']**

In [164]:
features_list = ['melspectrogram']

In [165]:
features = Extraction(train_data,
                      val_data,
                      window_size=4,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [01:14<00:00,  6.49it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:18<00:00,  7.18it/s]


344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step


In [166]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(10981,)

'melspectrogram'

(10981, 60)

In [167]:
train_y

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', ..., 'Anas strepera', 'Anas strepera',
       'Anas strepera'], dtype='<U25')

In [168]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(2928,)

'melspectrogram'

(2928, 60)

### Encode Classes

In [169]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [170]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

10981

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

2928

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [171]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [172]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[1.7294491e-05, 1.8239583e-05, 1.9485902e-05, ..., 5.1940111e-07,
          3.2287385e-08, 2.7940120e-08],
         [7.9314253e-07, 8.0714307e-07, 8.3096978e-07, ..., 1.3027967e-06,
          8.4759665e-07, 8.0172282e-07],
         [3.0612518e-06, 3.0607123e-06, 3.0592107e-06, ..., 6.0072955e-07,
          3.6999687e-08, 1.4126979e-08],
         ...,
         [1.0815166e-06, 1.0910663e-06, 1.1071655e-06, ..., 4.2367597e-07,
          1.3173792e-07, 1.2475495e-07],
         [2.1907644e-07, 2.7223422e-07, 3.6665881e-07, ..., 5.7446897e-07,
          1.8625977e-07, 1.7997159e-07],
         [2.4198832e-06, 2.6966893e-06, 3.1864115e-06, ..., 1.2659899e-05,
          2.3157411e-07, 2.2612821e-07]], dtype=float32),
  'label': array([1, 1, 1, ..., 5, 5, 5])},
 'val': {'melspectrogram': array([[3.75477839e-06, 3.76116282e-06, 3.77205765e-06, ...,
          5.65175878e-06, 5.06358674e-06, 4.80833251e-06],
         [3.40994302e-06, 3.41521968e-06, 3.42404178e-0

### Save the merged dictionary to a pkl

In [173]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_4s_mel.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc']**

In [174]:
features_list = ['melspectrogram', 'mfcc']

In [175]:
features = Extraction(train_data,
                      val_data,
                      window_size=4,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [04:23<00:00,  1.84it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:38<00:00,  3.39it/s]


344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step
344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 429us/step


In [176]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(10981,)

'melspectrogram'

(10981, 60)

'mfcc'

(10981, 20)

In [177]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(2928,)

'melspectrogram'

(2928, 60)

'mfcc'

(2928, 20)

### Encode Classes

In [178]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [179]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

10981

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

2928

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [180]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [181]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[1.7294491e-05, 1.8239583e-05, 1.9485902e-05, ..., 5.1940111e-07,
          3.2287385e-08, 2.7940120e-08],
         [7.9314253e-07, 8.0714307e-07, 8.3096978e-07, ..., 1.3027967e-06,
          8.4759665e-07, 8.0172282e-07],
         [3.0612518e-06, 3.0607123e-06, 3.0592107e-06, ..., 6.0072955e-07,
          3.6999687e-08, 1.4126979e-08],
         ...,
         [1.0815166e-06, 1.0910663e-06, 1.1071655e-06, ..., 4.2367597e-07,
          1.3173792e-07, 1.2475495e-07],
         [2.1907644e-07, 2.7223422e-07, 3.6665881e-07, ..., 5.7446897e-07,
          1.8625977e-07, 1.7997159e-07],
         [2.4198832e-06, 2.6966893e-06, 3.1864115e-06, ..., 1.2659899e-05,
          2.3157411e-07, 2.2612821e-07]], dtype=float32),
  'mfcc': array([[-1.0000000e+00, -2.3626368e-01, -5.0428110e-01, ...,
           5.1737577e-03,  1.1148715e-02,  3.4610374e-04],
         [-1.0000000e+00, -2.4848318e-01, -4.7058651e-01, ...,
           2.9826174e-03,  9.7121680e-03, -5.3395964e

### Save the merged dictionary to a pkl

In [182]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_4s_mel_mfcc.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma']**

In [183]:
features_list = ['melspectrogram', 'mfcc', 'chroma']

In [184]:
features = Extraction(train_data,
                      val_data,
                      window_size=4,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [04:09<00:00,  1.94it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:05<00:00,  1.98it/s]


344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step
344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 430us/step
344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step


In [185]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(10981,)

'melspectrogram'

(10981, 60)

'mfcc'

(10981, 20)

'chroma'

(10981, 12)

In [186]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(2928,)

'melspectrogram'

(2928, 60)

'mfcc'

(2928, 20)

'chroma'

(2928, 12)

### Encode Classes

In [187]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [188]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

10981

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

2928

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [189]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [190]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[1.7294491e-05, 1.8239583e-05, 1.9485902e-05, ..., 5.1940111e-07,
          3.2287385e-08, 2.7940120e-08],
         [7.9314253e-07, 8.0714307e-07, 8.3096978e-07, ..., 1.3027967e-06,
          8.4759665e-07, 8.0172282e-07],
         [3.0612518e-06, 3.0607123e-06, 3.0592107e-06, ..., 6.0072955e-07,
          3.6999687e-08, 1.4126979e-08],
         ...,
         [1.0815166e-06, 1.0910663e-06, 1.1071655e-06, ..., 4.2367597e-07,
          1.3173792e-07, 1.2475495e-07],
         [2.1907644e-07, 2.7223422e-07, 3.6665881e-07, ..., 5.7446897e-07,
          1.8625977e-07, 1.7997159e-07],
         [2.4198832e-06, 2.6966893e-06, 3.1864115e-06, ..., 1.2659899e-05,
          2.3157411e-07, 2.2612821e-07]], dtype=float32),
  'mfcc': array([[-1.0000000e+00, -2.3626368e-01, -5.0428110e-01, ...,
           5.1737577e-03,  1.1148715e-02,  3.4610374e-04],
         [-1.0000000e+00, -2.4848318e-01, -4.7058651e-01, ...,
           2.9826174e-03,  9.7121680e-03, -5.3395964e

### Save the merged dictionary to a pkl

In [191]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_4s_mel_mfcc_chroma.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma', 'rms']**

In [192]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'rms']

In [193]:
features = Extraction(train_data,
                      val_data,
                      window_size=4,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [04:53<00:00,  1.65it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:18<00:00,  1.66it/s]


344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step
344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step
344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step
344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step


In [194]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(10981,)

'melspectrogram'

(10981, 60)

'mfcc'

(10981, 20)

'chroma'

(10981, 12)

'rms'

(10981, 1)

In [195]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(2928,)

'melspectrogram'

(2928, 60)

'mfcc'

(2928, 20)

'chroma'

(2928, 12)

'rms'

(2928, 1)

### Encode Classes

In [196]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [197]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

10981

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

2928

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [198]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [199]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[1.7294491e-05, 1.8239583e-05, 1.9485902e-05, ..., 5.1940111e-07,
          3.2287385e-08, 2.7940120e-08],
         [7.9314253e-07, 8.0714307e-07, 8.3096978e-07, ..., 1.3027967e-06,
          8.4759665e-07, 8.0172282e-07],
         [3.0612518e-06, 3.0607123e-06, 3.0592107e-06, ..., 6.0072955e-07,
          3.6999687e-08, 1.4126979e-08],
         ...,
         [1.0815166e-06, 1.0910663e-06, 1.1071655e-06, ..., 4.2367597e-07,
          1.3173792e-07, 1.2475495e-07],
         [2.1907644e-07, 2.7223422e-07, 3.6665881e-07, ..., 5.7446897e-07,
          1.8625977e-07, 1.7997159e-07],
         [2.4198832e-06, 2.6966893e-06, 3.1864115e-06, ..., 1.2659899e-05,
          2.3157411e-07, 2.2612821e-07]], dtype=float32),
  'mfcc': array([[-1.0000000e+00, -2.3626368e-01, -5.0428110e-01, ...,
           5.1737577e-03,  1.1148715e-02,  3.4610374e-04],
         [-1.0000000e+00, -2.4848318e-01, -4.7058651e-01, ...,
           2.9826174e-03,  9.7121680e-03, -5.3395964e

### Save the merged dictionary to a pkl

In [200]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_4s_all.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

## **Window Size = 2s**

### **['melspectrogram']**

In [201]:
features_list = ['melspectrogram']

In [202]:
features = Extraction(train_data,
                      val_data,
                      window_size=2,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [01:29<00:00,  5.39it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:22<00:00,  5.81it/s]


708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step


In [203]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(22631,)

'melspectrogram'

(22631, 60)

In [204]:
train_y

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', ..., 'Anas strepera', 'Anas strepera',
       'Anas strepera'], dtype='<U25')

In [205]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(6044,)

'melspectrogram'

(6044, 60)

### Encode Classes

In [206]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [207]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

22631

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

6044

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [208]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [209]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[3.6285463e-05, 3.8200385e-05, 4.0739291e-05, ..., 3.0273466e-06,
          2.2592510e-06, 2.1416045e-06],
         [7.4347768e-09, 8.2490290e-09, 9.6704795e-09, ..., 2.0450429e-07,
          2.5535245e-08, 2.3674612e-08],
         [1.6406947e-06, 1.6677786e-06, 1.7138188e-06, ..., 2.2908193e-06,
          1.6840297e-06, 1.5936671e-06],
         ...,
         [2.7316248e-06, 2.9339224e-06, 3.2875062e-06, ..., 1.0139429e-06,
          5.8416873e-07, 5.6021997e-07],
         [2.4674987e-06, 2.5058257e-06, 2.5689174e-06, ..., 9.9417584e-06,
          2.3140782e-07, 1.8963851e-07],
         [3.0352521e-06, 4.1363774e-06, 6.0993666e-06, ..., 2.6265046e-05,
          7.4409758e-07, 7.3155365e-07]], dtype=float32),
  'label': array([1, 1, 1, ..., 5, 5, 5])},
 'val': {'melspectrogram': array([[4.3031432e-07, 4.3289390e-07, 4.3771132e-07, ..., 1.8346757e-06,
          1.3503952e-06, 1.2528141e-06],
         [1.0225300e-05, 1.0246177e-05, 1.0281189e-05, ..., 1

### Save the merged dictionary to a pkl

In [210]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_2s_mel.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc']**

In [211]:
features_list = ['melspectrogram', 'mfcc']

In [212]:
features = Extraction(train_data,
                      val_data,
                      window_size=2,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [03:17<00:00,  2.45it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:49<00:00,  2.63it/s]


708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 404us/step


In [213]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(22631,)

'melspectrogram'

(22631, 60)

'mfcc'

(22631, 20)

In [214]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(6044,)

'melspectrogram'

(6044, 60)

'mfcc'

(6044, 20)

### Encode Classes

In [215]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [216]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

22631

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

6044

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [217]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [218]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[3.6285463e-05, 3.8200385e-05, 4.0739291e-05, ..., 3.0273466e-06,
          2.2592510e-06, 2.1416045e-06],
         [7.4347768e-09, 8.2490290e-09, 9.6704795e-09, ..., 2.0450429e-07,
          2.5535245e-08, 2.3674612e-08],
         [1.6406947e-06, 1.6677786e-06, 1.7138188e-06, ..., 2.2908193e-06,
          1.6840297e-06, 1.5936671e-06],
         ...,
         [2.7316248e-06, 2.9339224e-06, 3.2875062e-06, ..., 1.0139429e-06,
          5.8416873e-07, 5.6021997e-07],
         [2.4674987e-06, 2.5058257e-06, 2.5689174e-06, ..., 9.9417584e-06,
          2.3140782e-07, 1.8963851e-07],
         [3.0352521e-06, 4.1363774e-06, 6.0993666e-06, ..., 2.6265046e-05,
          7.4409758e-07, 7.3155365e-07]], dtype=float32),
  'mfcc': array([[-1.0000000e+00, -2.3037937e-01, -5.0594997e-01, ...,
           2.8443993e-03,  1.4909296e-02,  8.9377072e-03],
         [-1.0000000e+00, -2.6351056e-01, -5.3618097e-01, ...,
           5.8985357e-03,  1.1296259e-02,  1.5974644e

### Save the merged dictionary to a pkl

In [219]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_2s_mel_mfcc.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma']**

In [220]:
features_list = ['melspectrogram', 'mfcc', 'chroma']

In [221]:
features = Extraction(train_data,
                      val_data,
                      window_size=2,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [07:06<00:00,  1.14it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:27<00:00,  1.49it/s]


708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step


In [222]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(22631,)

'melspectrogram'

(22631, 60)

'mfcc'

(22631, 20)

'chroma'

(22631, 12)

In [223]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(6044,)

'melspectrogram'

(6044, 60)

'mfcc'

(6044, 20)

'chroma'

(6044, 12)

### Encode Classes

In [224]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [225]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

22631

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

6044

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [226]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [227]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[3.6285463e-05, 3.8200385e-05, 4.0739291e-05, ..., 3.0273466e-06,
          2.2592510e-06, 2.1416045e-06],
         [7.4347768e-09, 8.2490290e-09, 9.6704795e-09, ..., 2.0450429e-07,
          2.5535245e-08, 2.3674612e-08],
         [1.6406947e-06, 1.6677786e-06, 1.7138188e-06, ..., 2.2908193e-06,
          1.6840297e-06, 1.5936671e-06],
         ...,
         [2.7316248e-06, 2.9339224e-06, 3.2875062e-06, ..., 1.0139429e-06,
          5.8416873e-07, 5.6021997e-07],
         [2.4674987e-06, 2.5058257e-06, 2.5689174e-06, ..., 9.9417584e-06,
          2.3140782e-07, 1.8963851e-07],
         [3.0352521e-06, 4.1363774e-06, 6.0993666e-06, ..., 2.6265046e-05,
          7.4409758e-07, 7.3155365e-07]], dtype=float32),
  'mfcc': array([[-1.0000000e+00, -2.3037937e-01, -5.0594997e-01, ...,
           2.8443993e-03,  1.4909296e-02,  8.9377072e-03],
         [-1.0000000e+00, -2.6351056e-01, -5.3618097e-01, ...,
           5.8985357e-03,  1.1296259e-02,  1.5974644e

### Save the merged dictionary to a pkl

In [228]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_2s_mel_mfcc_chroma.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma', 'rms']**

In [229]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'rms']

In [230]:
features = Extraction(train_data,
                      val_data,
                      window_size=2,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [08:21<00:00,  1.03s/it]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:41<00:00,  1.28it/s]


708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 393us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step


In [231]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(22631,)

'melspectrogram'

(22631, 60)

'mfcc'

(22631, 20)

'chroma'

(22631, 12)

'rms'

(22631, 1)

In [232]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(6044,)

'melspectrogram'

(6044, 60)

'mfcc'

(6044, 20)

'chroma'

(6044, 12)

'rms'

(6044, 1)

### Encode Classes

In [233]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [234]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

22631

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

6044

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [235]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [236]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[3.6285463e-05, 3.8200385e-05, 4.0739291e-05, ..., 3.0273466e-06,
          2.2592510e-06, 2.1416045e-06],
         [7.4347768e-09, 8.2490290e-09, 9.6704795e-09, ..., 2.0450429e-07,
          2.5535245e-08, 2.3674612e-08],
         [1.6406947e-06, 1.6677786e-06, 1.7138188e-06, ..., 2.2908193e-06,
          1.6840297e-06, 1.5936671e-06],
         ...,
         [2.7316248e-06, 2.9339224e-06, 3.2875062e-06, ..., 1.0139429e-06,
          5.8416873e-07, 5.6021997e-07],
         [2.4674987e-06, 2.5058257e-06, 2.5689174e-06, ..., 9.9417584e-06,
          2.3140782e-07, 1.8963851e-07],
         [3.0352521e-06, 4.1363774e-06, 6.0993666e-06, ..., 2.6265046e-05,
          7.4409758e-07, 7.3155365e-07]], dtype=float32),
  'mfcc': array([[-1.0000000e+00, -2.3037937e-01, -5.0594997e-01, ...,
           2.8443993e-03,  1.4909296e-02,  8.9377072e-03],
         [-1.0000000e+00, -2.6351056e-01, -5.3618097e-01, ...,
           5.8985357e-03,  1.1296259e-02,  1.5974644e

### Save the merged dictionary to a pkl

In [237]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_2s_all.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)